Over the summer I've been developing testing tools
for the developers and users
of the upcoming [Array API](https://data-apis.org/) standard.
Specifically I contributed "strategies" to the testing library [Hypothesis](https://github.com/HypothesisWorks/hypothesis/),
which I'm very excited to announce
is now available in [`hypothesis.extra.array_api`](https://hypothesis.readthedocs.io/en/latest/numpy.html#array-api)
since its recent version 6.21.0 release.

This blog post will be a gentle introduction
to how you could start using Hypothesis
to test your array-consuming functions.
I demonstrate a typical workflow
one would have
writing a generalised cumulative sums function—think [`np.cumsum()`](https://numpy.org/doc/stable/reference/generated/numpy.cumsum.html)
that would work for *all* [libraries adopting the Array API](https://data-apis.org/array-api/latest/purpose_and_scope.html#stakeholders).

### Before we begin

As well as requiring Hypothesis >= 6.21,
we need NumPy >= 1.22
so that we can test with its
[recently merged](https://github.com/numpy/numpy/pull/18585)
Array API implementation.
This hasn't been released yet,
so if you want to play around with it
I would recommend getting the [nightly builds](https://anaconda.org/scipy-wheels-nightly/numpy).

I will be using
the excellent [ipytest](https://github.com/chmp/ipytest/)
to nicely run tests
in Jupyter.
I also suppress all warnings
for convenience's sake.

In [1]:
%%capture
!pip install hypothesis>=6.21
!pip install -i https://pypi.anaconda.org/scipy-wheels-nightly/simple numpy

In [2]:
%%capture
!pip install ipytest
import ipytest; ipytest.autoconfig()

In [3]:
import warnings; warnings.filterwarnings("ignore")

### What the Array API enables

The [API](https://data-apis.org/array-api/latest/) standardises functionality of array libraries, which has [numerous benefits](https://data-apis.org/array-api/latest/use_cases.html) for both developers and users.
If you've used NumPy before you'll feel right at home!

The most exciting prospect for me
is being able to easily write an array-consuming method
that works all the adopting libraries.
Let's try writing this method
to calculate the cumulative sums of an array—

In [4]:
def cumsum(x):
    """Return the cumulative sum of the elements."""
    xp = x.__array_namespace__()
    
    result = xp.empty(x.size, dtype=x.dtype)
    result[0] = x[0]
    for i in range(1, x.size):
        result[i] = result[i - 1] + x[i]
        
    return result

The all-important
[`__array_namespace__()`](https://data-apis.org/array-api/latest/API_specification/array_object.html#method-array-namespace) method
allows array-consuming methods to get the array's respective Array API module.
Conventionally we assign it to the variable `xp`.

From there you just need to rely on the guarantees of the Array API
and you're suddenly supporting NumPy, TensorFlow, PyTorch, etc.
all in one simple method!

### Good ol' unit tests

I'm not the biggest fan of TDD [per-say](https://twitter.com/simonw/status/1424457164001669122),
but in any case I hope you'd want write some tests at some point.

We can import NumPy's Array API implementation
and test with that for now—in the future it'd be a good idea to try other implementations!.

In [5]:
from numpy import array_api as nxp

def test_cumsum():
    x = nxp.asarray([0, 1, 2, 3, 4])
    assert nxp.all(cumsum(x) == nxp.asarray([0, 1, 3, 6, 10]))
    
ipytest.run("-k cumsum")

.                                                                                            [100%]
1 passed in 0.01s


I would probably write a [parametrized](https://docs.pytest.org/en/stable/parametrize.html) test here
and write cases to cover all the interesting scenarios I can think of.
Whatever we do,
we will definitely miss some edge cases.
What if we could catch bugs
we would never think of ourselves?

### Testing our assumptions with Hypothesis

Hypothesis is a property-based testing library—I'll quote their excellent [docs](https://hypothesis.readthedocs.io/en/latest/index.html)
to summarise how it works.

> Think of a normal unit test as being something like the following:
> 1. Set up some data.
> 2. Perform some operations on the data.
> 3. Assert something about the result.
>
> Hypothesis lets you write tests which instead look like this:
> 1. For all data matching some specification.
> 2. Perform some operations on the data.
> 3. Assert something about the result.

You almost certainly will find new bugs with Hypothesis
thanks to how it cleverly fuzzes your specifications,
but the package really shines is how it ["reduces" failing test cases](https://drmaciver.github.io/papers/reduction-via-generation-preview.pdf)
to present only the minimal reproducer that trigger said bugs.
This demo will showcase both its power and user-friendliness.

Let's try testing a simple assumption that we can make about our `cumsum()` method:

> For an array with positive elements,
> its cumulative sums should only increment or remain the same per step.

<!--Formally we might express this assumption as $\forall i \in \{1,\ldots,\vert x \vert \}.f(x)_i - f(x)_{i-1} \geq 0$.-->
<!--Formally you might specify this assumption as
"if $A$ is a $n$-lengthed ordered set
containing values $v$ that satisfy $v\in\mathbb{R}$ and  $v\geq0$,
for the cumulative sums function $f$ defined as $f(A)_j = \sum_{i=1}^j A_i$,
when $j > 1$ the following is always true: $f(A)_j \geq f(A)_{j-1}$."-->

We can write a simple enough Hypothesis-powered test method for this—

In [6]:
from hypothesis import given
from hypothesis.extra.array_api import make_strategies_namespace

xps = make_strategies_namespace(nxp)

@given(xps.arrays(dtype="uint8", shape=10))
def test_cumsum_pos_arrays_accumulate(x):
    a = cumsum(x)
    assert nxp.all(a[1:] >= a[:-1])

As the Array API tools provided by Hypothesis
are agnostic to the adopting array/tensor libraries,
we first need to bind an implementation
via [`make_strategies_namespace()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#hypothesis.extra.array_api.make_strategies_namespace).
Passing `numpy.array_api` will give us
a [`SimpleNamespace`](https://docs.python.org/3/library/types.html#types.SimpleNamespace)
to use these tools for NumPy's Array API implementation.

The [`@given()`](https://hypothesis.readthedocs.io/en/latest/details.html#hypothesis.given) decorator
tells Hypothesis what values it should generate for our test method.
In this case
[`xps.arrays()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#xps.arrays) is a "search strategy" 
that specifies Array API-compliant arrays from `numpy.array_api`
should be generated.

In this case,
`shape=10` specifies the arrays generated are 1-dimensional and of size 10,
and `dtype="uint8`  specifies they should contain unsigned integers
(which is handy for our test method as uints are always positive).
Let's quickly see a small sample of the arrays Hypothesis can generate—

In [7]:
for _ in range(10):
    x = xps.arrays(dtype="uint8", shape=10, unique=True).example()
    print(x)

[ 51 118  63 162 133 177 185  28 156 115]
[123  84 122 142  54  26 244 129 220 253]
[  0 255 254 253 252 251 250 249 248 247]
[ 99  63  60  75 148 171 174 153 255 198]
[203   0 254 253 252 251 250 249 248 247]
[118  31  82 227 216 186   3 101 173   7]
[198 174  70 253 177  97 254   8 202  40]
[103   0 254 253 252 251 250 249 248 247]
[152 135   5 211 121 237 182 243 128 104]
[160   0 254 253 252 251 250 249 248 247]


How Hypothesis "draws" from its strategies can look rather unremarkable at first.
A small sample of draws might look fairly uniform
but trust that strategies will end up covering all kinds of edge cases.
Importantly it will cover these cases effeciently
so that Hypothesis-powered tests are relatively quick to run on your machine.

All our test method does is get the cumulative sums array `a`
that is returned from `cumsum(x)`,
and then check that every element `a[i]`
is greater than or equal to `a[i-1]`.

Time to run it!

In [8]:
ipytest.run("-k cumsum_pos_arrays_accumulate", "--hypothesis-seed=3")

F                                                                                            [100%]
============================================= FAILURES =============================================
________________________________ test_cumsum_pos_arrays_accumulate _________________________________

    @given(xps.arrays(dtype="uint8", shape=10))
>   def test_cumsum_pos_arrays_accumulate(x):

<cell>:7: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

x = Array([26, 26, 26, 26, 26, 26, 26, 26, 26, 26], dtype=uint8)

    @given(xps.arrays(dtype="uint8", shape=10))
    def test_cumsum_pos_arrays_accumulate(x):
        a = cumsum(x)
>       assert nxp.all(a[1:] >= a[:-1])
E       assert Array(False, dtype=bool)
E        +  where Array(False, dtype=bool) = <function all at 0x7fccd95528b0>(Array([ 52,  78, 104, 130, 156, 182, 208, 234,   4], dtype=uint8) >= Array([ 26,  52,  78, 104, 130, 156, 182, 208, 234], dtype=uint8))
E        +   

Hypothesis has tested our assumption
and told us we're wrong!
It should provide us with the following falsifying example:

```python
>>> x = xp.full(10, 26, dtype=xp.uint8)
>>> x
Array([ 26,  26,  26,  26,  26,  26,  26,  26,  26,  26], dtype=uint8)
>>> cumsum(x)
Array([ 26,  52,  78, 104, 130, 156, 182, 208, 234,   4], dtype=uint8)
```

You can see that an overflow error has occured for the final cumulative sum,
as 234 + 26 (260) cannot be represented in 8-bit unsigned integers.

Let's try promoting the dtype of the cumulative sums array
so that it can represent larger numbers,
and then we can run the test again.

In [9]:
def max_dtype(xp, dtype):
    if dtype in [getattr(xp, name) for name in ("int8", "int16", "int32", "int64")]:
        return xp.int64
    elif dtype in [getattr(xp, name) for name in ("uint8", "uint16", "uint32", "uint64")]:
        return xp.uint64
    else:
        return xp.float64
    
def cumsum(x):
    xp = x.__array_namespace__()
    
    result = xp.empty(x.size, dtype=max_dtype(xp, x.dtype))
    result[0] = x[0]
    for i in range(1, x.size):
        result[i] = result[i - 1] + x[i]
        
    return result

ipytest.run("-k cumsum_uint8_arrays_accumulate")


2 deselected in 0.00s


You can see another assumption about our code is:

> We can find the cumulative sums of arrays of any scalar dtype.

We should cover this assumption in our test method `test_cumsum_pos_arrays_accumulate`
by passing child search strategies
into our [`xps.arrays()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#xps.arrays) parent strategy.
Specifying `dtype` as [`xps.scalar_dtypes()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#xps.scalar_dtypes)
will tell Hypothesis to generate arrays of all scalar dtypes.
To specify that these array values should be positive,
we can just pass keyword arguments to the underlying
value generating strategy [`xps.from_dtype()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#xps.from_dtype)
via `elements={"min_value": 0}`.

And while we're at it, let's make sure to cover another assumption:

> We can find the cumulative sums of arrays with multiple dimensions.

Specifying `shape` as [`xps.array_shapes()`](https://hypothesis.readthedocs.io/en/latest/numpy.html#xps.array_shapes)
will tell Hypothesis to generate arrays of various dimensionality and sizes.
We can [filter](https://hypothesis.readthedocs.io/en/latest/data.html#filtering)
this strategy with `lambda s: prod(s) > 1`
so that always `x.size > 1`
(to allow our test code to work).

In [10]:
from math import prod
from hypothesis import settings

@given(
    xps.arrays(
        dtype=xps.scalar_dtypes(),
        shape=xps.array_shapes().filter(lambda s: prod(s) > 1),
        elements={"min_value": 0},
    )
)
def test_cumsum_pos_arrays_accumulate(x):
    a = cumsum(x)
    assert nxp.all(a[1:] >= a[:-1])
    
ipytest.run("-k cumsum_pos_arrays_accumulate", "--hypothesis-seed=3")

F                                                                                            [100%]
============================================= FAILURES =============================================
________________________________ test_cumsum_pos_arrays_accumulate _________________________________

    @given(
>       xps.arrays(
            dtype=xps.scalar_dtypes(),
            shape=xps.array_shapes().filter(lambda s: prod(s) > 1),
            elements={"min_value": 0},
        )
    )
E   hypothesis.errors.MultipleFailures: Hypothesis found 2 distinct failures.

<cell>:5: MultipleFailures
-------------------------------------------- Hypothesis --------------------------------------------
Falsifying example: test_cumsum_pos_arrays_accumulate(
    x=Array([[False, False]], dtype=bool),
)
TypeError: only size-1 arrays can be converted to Python scalars

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<cell>", line 12,

Again Hypothesis has proved our assumptions wrong.
This time it's found two problems.

Firstly, our `cumsum()` method doesn't adjust for boolean arrays,
so we get an error when we blindly add two `bool` values together.

```python
>>> x = xp.zeros(2, dtype=xp.bool)
>>> x
Array([False, False], dtype=bool)
>>> cumsum(x)
Traceback:
  <cell>, line 15, in cumsum
    result[i] = result[i - 1] + x[i]
  ...
TypeError: Only numeric dtypes are allowed in __add__
```
   
Secondly, our `cumsum()` method is assuming arrays are 1-dimensional,
so we get an error when we blindly
assume`x[0]` will always return a single scalar
(technically a 0-dimensional array).

```python
>>> x = xp.zeros((1, 2), dtype=xp.bool)
>>> x
Array([[False, False]], dtype=bool)
>>> cumsum(x)
Traceback:
  <cell>, line 13, in cumsum
    result[0] = x[0]
  ...
TypeError: only size-1 arrays can be converted to Python scalars
```

So I'm going to
flatten input arrays
and convert the boolean arrays to integer arrays of `0` and `1`.
Of-course we'll run the test again to make sure our updated `cumsum()` method now works.

In [11]:
def cumsum(x):
    xp = x.__array_namespace__()
    
    x = xp.reshape(x, x.size)
    
    if x.dtype == xp.bool:
        mask = x
        dtype = xp.uint64
        x = xp.zeros(x.shape, dtype=xp.uint64)
        x[mask] = 1
        
    result = xp.empty(x.size, dtype=max_dtype(xp, x.dtype))
    result[0] = x[0]
    for i in range(1, x.size):
        result[i] = result[i - 1] + x[i]
        
    return result

ipytest.run("-k cumsum_pos_arrays_accumulate", "--hypothesis-seed=3")

F                                                                                            [100%]
============================================= FAILURES =============================================
________________________________ test_cumsum_pos_arrays_accumulate _________________________________

    @given(
>       xps.arrays(
            dtype=xps.scalar_dtypes(),
            shape=xps.array_shapes().filter(lambda s: prod(s) > 1),
            elements={"min_value": 0},
        )
    )

<cell>:5: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

x = Array([4611686018427387904, 4611686018427387904], dtype=int64)

    @given(
        xps.arrays(
            dtype=xps.scalar_dtypes(),
            shape=xps.array_shapes().filter(lambda s: prod(s) > 1),
            elements={"min_value": 0},
        )
    )
    def test_cumsum_pos_arrays_accumulate(x):
        a = cumsum(x)
>       assert nxp.all(a[1:] >= a[:-1])
E       assert Arra

We resolved our two previous issues
but Hypothesis has found yet another failing scenario!

```python
        x=Array([ 4611686018427387904,  4611686018427387904], dtype=int64)
cumsum(x)=Array([ 4611686018427387904, -9223372036854775808], dtype=int64)
```

Ah, back to overflows.
There's not much we can do about this
as there's no larger signed integer dtype than `int64`,
so we'll just have `cumsum()` detect overflows itself.
Overflow behaviour is actually not specified by the Array API,
so it could be that an implementing library will raise an error before we do anyway.

In [12]:
def cumsum(x):
    xp = x.__array_namespace__()
    
    x = xp.reshape(x, x.size)
    
    if x.dtype == xp.bool:
        mask = x
        dtype = xp.uint64
        x = xp.zeros(x.shape, dtype=xp.uint64)
        x[mask] = 1
        
    result = xp.empty(x.size, dtype=max_dtype(xp, x.dtype))
    result[0] = x[0]
    for i in range(1, x.size):
        result[i] = result[i - 1] + x[i]
        if result[i] < result[i - 1]:
            raise OverflowError("Cumulative sum cannot be represented")
        
    return result

If Hypothesis generates arrays which raise `OverflowError`,
we can just catch it
and use [`assume(False)`](https://hypothesis.readthedocs.io/en/latest/details.html#making-assumptions)
to ignore testing these arrays on runtime.
We can explicitly cover overflows in a seperate test.

In [13]:
from hypothesis import assume
import pytest

@given(
    xps.arrays(
        dtype=xps.scalar_dtypes(),
        shape=xps.array_shapes().filter(lambda s: prod(s) > 1),
        elements={"min_value": 0},
    )
)
def test_cumsum_pos_arrays_accumulate(x):
    try:
        a = cumsum(x)
        assert nxp.all(a[1:] >= a[:-1])
    except OverflowError:
        assume(False)
    
def test_cumsum_fails_on_overflow():
    x = nxp.asarray([nxp.iinfo(nxp.uint64).max, 1], dtype=nxp.uint64)
    with pytest.raises(OverflowError):
        cumsum(x)
        
ipytest.run("-k cumsum_pos_arrays_accumulate", "-k cumsum_fails_on_overflow")

.                                                                                            [100%]
1 passed, 2 deselected in 0.01s


So there we have it TODO